<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH17_04_%ED%95%9C%EA%B5%AD%EC%96%B4_BERT%EC%9D%98_%EB%A7%88%EC%8A%A4%ED%81%AC%EB%93%9C_%EC%96%B8%EC%96%B4_%EB%AA%A8%EB%8D%B8_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.1 MB/s eta 0:00:00


### 1. 마스크드 언어 모델과 토크나이저
------------------------
transformers 패키지를 사용하여 모델과 토크나이저를 로드함. 

klue/bert-base는 대표적인 한국어 BERT임. klue/bert-base의 마스크드 언어모델과 klue/bert-base의 토크나이저를 로드함

In [2]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [4]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True) # pyTorch로 학습된 모델이지만 텐서플로우에서 사용하기 때문에 from_pt=True
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


### 2. BERT의 입력
------------------
예시 : '축구는 정말 재미있는 [MASK]다'

In [5]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

토크나이저로 변환된 결과에서 input_ids를 통해 정수 인코딩 결과를 확인할 수 있음

In [6]:
print(inputs['input_ids'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


토크나이저로 변환된 결과에서 token_type_ids를 통해 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수 있음

In [7]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


입력문장이 한개이므로 문장길이만큼 0 시퀀스를 얻음. 토크나이저로 변환된 결과에서 attention_mask를 통해 실제 단어와 패팅 토큰 구분을 확인할 수 있음

In [8]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


현재 입력에서 패딩이 없기 때문에, 문장길이 만큼의 1 시퀀스를 얻음


### 3. [MASK] 토큰 예측하기
--------------------
FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여줌. FillMaskPipeline에 우선 앞서 불러온 모델과 토크나이저를 지정해 줌.

In [9]:
from transformers import FillMaskPipeline

pipe = FillMaskPipeline(model=model, tokenizer=tokenizer)

예시로 부터 [MASK] 위치에 들어갈 수 있는 상위 5개 후보 단어를 출력함

In [10]:
pipe('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963514566421509,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.02595745027065277,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033912025392056,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924332283437252,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844172418117523,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [11]:
pipe('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.8382409811019897,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.028275717049837112,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.017189353704452515,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.014989719726145267,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.009382645599544048,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]

In [12]:
pipe('나는 오늘 아침에 [MASK]에 출근을 했다.')

[{'score': 0.08012557774782181,
  'token': 3769,
  'token_str': '회사',
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.'},
 {'score': 0.06124049797654152,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '나는 오늘 아침에 에 출근을 했다.'},
 {'score': 0.01748667098581791,
  'token': 4345,
  'token_str': '공장',
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.'},
 {'score': 0.016131814569234848,
  'token': 5841,
  'token_str': '사무실',
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.'},
 {'score': 0.015360799618065357,
  'token': 3671,
  'token_str': '서울',
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.'}]